In [1]:
%cd ..
%reload_ext autoreload
%autoreload 2

/Users/anantabhattarai/Documents/TUM/ADLR/tum-adlr-ws20-9


# Import Libraries

In [2]:
from alpha_one.game.information_set import InformationSetGenerator
import numpy as np
import pyspiel

from open_spiel.python.algorithms import mcts
from open_spiel.python.algorithms.alpha_zero import evaluator as evaluator_lib
from open_spiel.python.algorithms.alpha_zero import model as model_lib
from open_spiel.python.algorithms.mcts import SearchNode

from alpha_one.alg.imperfect_information import AlphaOneImperfectInformationMCTSEvaluator
from alpha_one.alg.imperfect_information import DeterminizedMCTSEvaluator
from alpha_one.alg.mcts import ImperfectInformationMCTSBot

from alpha_one.utils.statemask import get_state_mask
from open_spiel.python.algorithms import get_all_states


from alpha_one.game import trajectory
from alpha_one.game import buffer

# Load game and define hyperparameters

In [3]:
uct_c = 3                                     
max_simulations = 100   
verbose = False
policy_epsilon = 0.25                             
policy_alpha = 1
temperature_drop = 20
temperature = 0.5

In [4]:
# state_to_value maps each possible state in the game to the value
# for eg: intial state can be mapped as 1 and final state can be mapped as 53

game_name = "leduc_poker"
game = pyspiel.load_game(game_name)
states = get_all_states.get_all_states(game)
state_to_value = {}
a = 0
for key, values in states.items():
    state_to_value[values.__str__()] = a
    a += 1

# Initialize Neural Network Model

In [ ]:
model_saves_path = '../model_saves/kuhn_poker/obs_node_model'
nn_width = 50
nn_depth = 5
learning_rate = 0.001
weight_decay = 0.0001

model_type = 'mlp'
    
obs_model = model_lib.Model.build_model(
                              model_type, len(state_to_value), len(state_to_value),
                              nn_width=nn_width, nn_depth=nn_depth, 
                              weight_decay=weight_decay, 
                              learning_rate=learning_rate, 
                              path=model_saves_path)

In [ ]:
model_saves_path = '../model_saves/kuhn_poker/game_node_model'
nn_width = 50
nn_depth = 5
learning_rate = 0.001
weight_decay = 0.0001

model_type = 'mlp'

print(game.num_distinct_actions())
    
game_model = model_lib.Model.build_model(
                              model_type, game.observation_tensor_shape(), game.num_distinct_actions(),
                              nn_width=nn_width, nn_depth=nn_depth, 
                              weight_decay=weight_decay, 
                              learning_rate=learning_rate, 
                              path=model_saves_path)

# Define Required Functions

In [ ]:
evaluator = AlphaOneImperfectInformationMCTSEvaluator(state_to_value, obs_model, game_model)

In [ ]:
# get policy and value at the observation node
def get_policy_value_obs_node(root, state_mask, index_track):
    
    # state_mask and state_masked_policy are used while training NN

    policy = np.zeros(len(root.children))
    for c in root.children:
        #policy[c.action] = c.explore_count
        if c.explore_count == 0:
            policy[c.action] += c.total_reward
        else:
            policy[c.action] += c.total_reward / c.explore_count
    
    policy = (policy - policy.min()) / (policy.max() - policy.min())
    policy /= policy.sum()

    state_masked_policy = np.zeros(len(state_mask))
    
    for i in range(len(index_track)):
        state_masked_policy[index_track] = policy[i]
        
    return state_masked_policy, policy

In [ ]:
# get policy and value at the game node after guessing the state
def get_policy_value_game_node(root, guess_state):
    
    game_name = "leduc_poker"
    game_temp = pyspiel.load_game(game_name)
    policy = np.zeros(game_temp.num_distinct_actions())
    
    for c in root.children[guess_state].children:
        #policy[c.action] = c.explore_count
        if c.explore_count == 0:
            policy[c.action] += c.total_reward
        else:
            policy[c.action] += c.total_reward / c.explore_count
    
    policy = (policy - policy.min()) / (policy.max() - policy.min())
    policy /= policy.sum()
    
    return policy

In [ ]:
def ii_mcts_agent(information_set_generator, state_to_value, ii_mcts_bot):
    root, _ = ii_mcts_bot.mcts_search(information_set_generator)
    
    information_set = information_set_generator.calculate_information_set()
    
    state_mask, index_track = get_state_mask(state_to_value, information_set)
    
    state_masked_policy, state_policy = get_policy_value_obs_node(root, state_mask, index_track)
    
    guess_state = np.argmax(state_policy)
    
    game_node_policy = get_policy_value_game_node(root, guess_state)

    return state_masked_policy, game_node_policy, information_set[guess_state], state_mask

In [ ]:
def play_one_game():
    
    # trajectory of the observation nodes for training NN
    trajectory_obs = trajectory.GameTrajectory()
    
    # trajectory of the game nodes for training NN
    trajectory_game = trajectory.GameTrajectory()
    
    actions = []
    
    
    state = game.new_initial_state()
    information_set_generator = InformationSetGenerator(game)
    
    ii_mcts_bot = ImperfectInformationMCTSBot(game,
                                            uct_c,
                                                  max_simulations,
                                                  evaluator,
                                                  False,
                                                  child_selection_fn=SearchNode.puct_value)
    while not state.is_terminal():
        
        if state.current_player() < 0:
            action = np.random.choice(state.legal_actions())

            information_set_generator.register_action(action)
            state.apply_action(action)
            information_set_generator.register_observation(state)
            
            
        else:
            
            #observations = information_set_generator._get_observation(state, state.current_player())
            
            state_masked_policy, game_node_policy, guess_state, state_mask = ii_mcts_agent(
                                                                     information_set_generator, 
                                                                     state_to_value, 
                                                                     ii_mcts_bot)
            
            #if len(actions) > temperature_drop:
            #    action = np.argmax(game_node_policy)
            #else:
            action = np.random.choice(len(game_node_policy), p=game_node_policy)
            
            actions.append(action)
            
            trajectory_obs.states.append(trajectory.TrajectoryState(state_mask, 
                                                                    state.current_player(), 
                                                                    state_mask, 
                                                                    action,
                                                                    state_masked_policy))
                                         
            trajectory_game.states.append(trajectory.TrajectoryState(guess_state.observation_tensor(), 
                                                                    state.current_player(), 
                                                                    state.legal_actions_mask(), 
                                                                    action,
                                                                    game_node_policy))
            
            
            information_set_generator.register_action(action)
            state.apply_action(action)
            information_set_generator.register_observation(state)
            
            
    trajectory_obs.returns = state.returns()
    trajectory_game.returns = state.returns()
    return trajectory_obs, trajectory_game

In [ ]:
def collect_trajectories(replay_buffer_obs, replay_buffer_game):
    
    for j in range(30):
        trajectory_obs, trajectory_game = play_one_game()
        
        p1_outcome = trajectory_obs.returns[0]
        
        
        replay_buffer_obs.extend(model_lib.TrainInput(s.observation, s.legals_mask, s.policy, p1_outcome) 
                             for s in trajectory_obs.states)
        
        replay_buffer_game.extend(model_lib.TrainInput(s.observation, s.legals_mask, s.policy, p1_outcome) 
                             for s in trajectory_game.states)

In [ ]:
replay_buffer_obs = buffer.ReplayBuffer(100)
replay_buffer_game = buffer.ReplayBuffer(100)

# Train AlphaOne

In [ ]:
for i in range(11):
    losses_obs = []
    losses_game = []
    
    collect_trajectories(replay_buffer_obs, replay_buffer_game)
    
    for _ in range(100 // 20):
        data_obs = replay_buffer_obs.sample(20)
        data_game = replay_buffer_game.sample(20)
        
        
        losses_obs.append(obs_model.update(data_obs))
        losses_game.append(game_model.update(data_game))
        
    losses = sum(losses_obs, model_lib.Losses(0, 0, 0)) / len(losses_obs)
        
    print(f"Loss at iteration {i} of observation model: {losses}")
        
    losses = sum(losses_game, model_lib.Losses(0, 0, 0)) / len(losses_game)
        
    print(f"Loss at iteration {i} of game model: {losses}")
    
    print("")

# Train Determinized MCTS

In [5]:
model_saves_path = '../model_saves/kuhn_poker/determinized_model'
nn_width = 50
nn_depth = 5
learning_rate = 0.001
weight_decay = 0.0001

model_type = 'mlp'

print(game.num_distinct_actions())
    
determinized_model = model_lib.Model.build_model(
                              model_type, game.observation_tensor_shape(), game.num_distinct_actions(),
                              nn_width=nn_width, nn_depth=nn_depth, 
                              weight_decay=weight_decay, 
                              learning_rate=learning_rate, 
                              path=model_saves_path)

3
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
def initialize_bot(game, uct_c, max_simulations, policy_epsilon, policy_alpha):
    
    if policy_epsilon == None or policy_alpha == None:
        noise = None
    else:
        noise = (policy_epsilon, policy_alpha)
    
    _evaluator = DeterminizedMCTSEvaluator(determinized_model)

    bot = mcts.MCTSBot(
          game,
          uct_c,
          max_simulations,
          _evaluator,
          solve=False,
          dirichlet_noise=noise,
          child_selection_fn=mcts.SearchNode.puct_value,
          verbose=False)
    
    return bot

In [7]:
def mcts_agent(state, information_set_generator):
    current_player = state.current_player()
    information_set = information_set_generator.calculate_information_set(current_player)
    policy = np.zeros(game.num_distinct_actions())

    # Evaluate each state in the information set by MCTS independently.
    # After the searches are completed, the numbers of visits for each action from the root 
    # are summed across all trees, 
    # and an action is chosen that maximises the total number of visits.
    for s in information_set:
        bot = initialize_bot(game, uct_c, max_simulations, policy_epsilon, policy_alpha)
        root = bot.mcts_search(s)
        if verbose:
            print_game_tree(root)
        for c in root.children:
            #policy[c.action] += c.explore_count
            #if c.explore_count == 0:
            #    policy[c.action] += c.total_reward
            #else:
            policy[c.action] += c.explore_count
    if temperature != 0:
        policy = policy ** (1 / temperature)
    policy /= policy.sum()
    return policy

In [8]:
def play_one_game_determinized_MCTS():

    trajectory_game = trajectory.GameTrajectory()
    
    actions = []
    
    state = game.new_initial_state()
    information_set_generator = InformationSetGenerator(game)
    
    while not state.is_terminal():
        
        if state.current_player() < 0:
            action = np.random.choice(state.legal_actions())

            information_set_generator.register_action(action)
            state.apply_action(action)
            information_set_generator.register_observation(state)
            
            
        else:
            
            policy = mcts_agent(state, information_set_generator)
            
            if temperature_drop == None:
                action = np.argmax(policy)
            elif len(actions) < temperature_drop:
                action = np.random.choice(len(policy), p=policy)
            else:
                action = np.argmax(policy)
            
            actions.append(action)
            
            trajectory_game.states.append(trajectory.TrajectoryState(state.observation_tensor(), 
                                                                    state.current_player(), 
                                                                    state.legal_actions_mask(), 
                                                                    action,
                                                                    policy))
            
            
            information_set_generator.register_action(action)
            state.apply_action(action)
            information_set_generator.register_observation(state)
            
            
    trajectory_game.returns = state.returns()
    return trajectory_game
    

In [9]:
def collect_trajectories(replay_buffer):
    
    for j in range(30):
        trajectory_game = play_one_game_determinized_MCTS()
        
        p1_outcome = trajectory_game.returns[0]
        
        
        replay_buffer.extend(model_lib.TrainInput(s.observation, s.legals_mask, s.policy, p1_outcome) 
                             for s in trajectory_game.states)

In [10]:
replay_buffer = buffer.ReplayBuffer(100)

In [11]:
for i in range(15):
    losses = []
    
    collect_trajectories(replay_buffer)
    
    for _ in range(100 // 20):
        data = replay_buffer.sample(20)
        
        
        losses.append(determinized_model.update(data))
        
    losses = sum(losses, model_lib.Losses(0, 0, 0)) / len(losses)
        
    print(f"Loss at iteration {i} of determinized model: {losses}")
    
    print("")

Loss at iteration 0 of determinized model: Losses(total: 27.671, policy: 0.779, value: 26.876, l2: 0.016)

Loss at iteration 1 of determinized model: Losses(total: 29.054, policy: 0.693, value: 28.346, l2: 0.016)



KeyboardInterrupt: 

# Evaluations

In [ ]:
game_returns = []
for _ in range(100):
    actions = []
    state = game.new_initial_state()
    information_set_generator = InformationSetGenerator(game)
    
    ii_mcts_bot = ImperfectInformationMCTSBot(game,
                                            uct_c,
                                                  max_simulations,
                                                  evaluator,
                                                  False,
                                                  child_selection_fn=SearchNode.puct_value)
    
    while not state.is_terminal():

        # environment state
        if state.current_player() < 0:
            action = np.random.choice(state.legal_actions())

            action_str = state.action_to_string(state.current_player(), action)
            actions.append(action_str)

            information_set_generator.register_action(action)
            state.apply_action(action)
            information_set_generator.register_observation(state)

        # player 1 as a MCTS bot
        elif state.current_player() == 0:
            policy = mcts_agent(state, information_set_generator)
            action = np.argmax(policy)

            action_str = state.action_to_string(state.current_player(), action)
            actions.append(action_str)

            information_set_generator.register_action(action)
            state.apply_action(action)
            information_set_generator.register_observation(state)

        # player 2 as IIG-MCTS bot
        else:
            # action = np.random.choice(state.legal_actions())
            
            state_masked_policy, game_node_policy, guess_state, state_mask = ii_mcts_agent(
                                                                     information_set_generator, 
                                                                     state_to_value, 
                                                                     ii_mcts_bot)
            action = np.argmax(game_node_policy)

            action_str = state.action_to_string(state.current_player(), action)
            actions.append(action_str)

            information_set_generator.register_action(action)
            state.apply_action(action)
            information_set_generator.register_observation(state)


    #print(actions)       
    #print(state.returns())
    #print()
    game_returns.append(state.returns())

In [ ]:
game_returns = np.array(game_returns)
print(f"Average return: {game_returns.mean(axis=0)}")